In [11]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe

import ro.jf.bk.account.sdk.AccountSdk
import ro.jf.bk.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import java.util.UUID.randomUUID

val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val ACCOUNT_MATCHERS_CSV_FILE = "../../data/provision/account-matchers.csv"
val FUND_MATCHERS_CSV_FILE = "../../data/provision/fund-matchers.csv"

// TODO(Johann) create a user
val userId = randomUUID()

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

In [3]:
import ro.jf.bk.account.api.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(userId)
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateCurrencyAccountTO(row.getValue<String>("account"), row.getValue<String>("currency")) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(userId, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

Currency(id=c779fb38-d63e-40c2-a420-925d122667e4, name=Cash RON, currency=Cash RON)
Currency(id=1974cd52-38dd-42ed-8ad1-2f2e2b731dcc, name=ING RON, currency=RON)
Currency(id=7337e0cd-e0b3-42ad-9489-bee268a96acc, name=ING Economy, currency=RON)
Currency(id=ce37cdce-8003-42fc-b893-491fb84ba279, name=Food Coupons, currency=RON)

In [5]:
import ro.jf.bk.fund.api.model.*

val funds = runBlocking {
    // TODO(Johann) fix npe on get existing funds without accounts
    val existingFunds = fundSdk.listFunds(userId)
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(row.getValue<String>("fund"), emptyList()) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(userId, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=09e168cf-9eb9-4b51-952f-698d97e70495, name=Expenses, accounts=[])
FundTO(id=0c9e790f-b648-4623-92d6-bd0dbf9f0684, name=Programming, accounts=[])
FundTO(id=cba87cb6-505d-4b4b-b45f-3ebe4228dbf6, name=Savings, accounts=[])

In [6]:
import ro.jf.funds.importer.api.model.AccountMatcherTO

val accountMatchers = runBlocking {
    DataFrame.readCSV(ACCOUNT_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row -> AccountMatcherTO(row.getValue<String>("import_account_name"),row.getValue<String>("account_name")) }
}
accountMatchers.map { it.toString() }.joinToString(separator = "\n")

AccountMatcherTO(importAccountName=ING old, accountName=ING RON)
AccountMatcherTO(importAccountName=ING Economy old, accountName=ING Economy)
AccountMatcherTO(importAccountName=Cash RON, accountName=Cash RON)
AccountMatcherTO(importAccountName=Food Coupons, accountName=Food Coupons)

In [15]:
import ro.jf.funds.importer.api.model.*

val fundMatchers = runBlocking {
    DataFrame.readCSV(FUND_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            when (row.getValue<String>("type")) {
                "by_label" -> FundMatcherTO.ByLabel(
                    importLabel = row.getValue<String>("import_label"),
                    fundName = row.getValue<String>("fund_name")
                )

                "by_account_label" -> FundMatcherTO.ByAccountLabel(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    fundName = row.getValue<String>("fund_name")
                )

                "by_account_label_with_transfer" -> FundMatcherTO.ByAccountLabelWithTransfer(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    initialFundName = row.getValue<String>("initial_fund_name"),
                    fundName = row.getValue<String>("fund_name")
                )

                else -> error("fund matcher type not recognized ${row.getValue<String>("type")}")
            }
        }
}
fundMatchers.map { it.toString() }.joinToString(separator = "\n")

ByLabel(importLabel=Basic - Food, fundName=Expenses)
ByLabel(importLabel=C&T - Gas & Parking, fundName=Expenses)
ByLabel(importLabel=Development - Education, fundName=Expenses)
ByLabel(importLabel=F&V - Fun, fundName=Expenses)
ByLabel(importLabel=S&S - Objects, fundName=Expenses)
ByLabel(importLabel=Home - House, fundName=Expenses)
ByLabel(importLabel=Gifts, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=ING Economy Old, importLabel=Invest Profit Acc, initialFundName=Savings, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=Food Coupons, importLabel=Work Income, initialFundName=Programming, fundName=Expenses)

In [17]:
import kotlinx.coroutines.runBlocking
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*

val csvFile = File("../../data/wallet/2019/wallet_export_2019_01.csv")
val importConfiguration = ImportConfigurationTO(
    fileType = ImportFileTypeTO.WALLET_CSV,
    accountMatchers = accountMatchers,
    fundMatchers = fundMatchers
)
// TODO(Johann) fix npe on import
val response = runBlocking { importSdk.import(userId, csvFile, importConfiguration) }
response

ImportResponse(response=Imported in service)